In [60]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import json


In [74]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get("https://www.loopnet.com/")

# Page load time. Can use time.sleep or WebDriverWait for dynamic content)
time.sleep(2)

In [75]:
# For sale or lease
search_for_sale = driver.find_element(By.CLASS_NAME, "search-types--for-sale")
search_for_sale.click()

# Property type
search_property_type = driver.find_element(By.NAME, "propertytypeforsale")
search_property_type.send_keys("Office")

# NOTE TO SELF: A better approach would be to navigate the dropdown menu and select the options

# Location
search_box = driver.find_element(By.NAME, "geography")
search_box.send_keys("Agoura Hills, CA")

search_box.send_keys(Keys.ENTER)
time.sleep(5)

In [80]:
# Extracting data (e.g., property titles)

# search_results = driver.find_elements(By.CLASS_NAME, "placards")

result = driver.find_element(By.CLASS_NAME, "placards")
ul_elements = result.find_elements(By.XPATH, './ul') 

print(len(ul_elements))

properties = []
for ul in ul_elements:
    li_elements = ul.find_elements(By.TAG_NAME, "li")
    print(len(li_elements))
    for li in li_elements:
        # Extract the desired information from each li element
        property_info = li.text
        properties.append({"property_info": property_info})

# Store data in a JSON file
with open('properties.json', 'w') as json_file:
    json.dump(properties, json_file, indent=4)

    

# Close the browser

# driver.quit()

2


In [64]:
# # Convert the data into a Pandas DataFrame
# df = pd.DataFrame(titles, columns=["Property Title"])

# # Display the DataFrame
# df.head()

# # Optional: Save the data to a CSV file
# df.to_csv("loopnet_property_titles.csv", index=False)